### Compute structure factor for $N=100$

$$ I(k) = \sum_{i=0}^N \sum_{j=0}^N \left\langle \frac{\sin{k ||\vec{R}_i - \vec{R}_j||}}{k ||\vec{R}_i - \vec{R}_j||} \right\rangle$$

In [1]:
# Library for reading simulation data
import numpy as np

# Parameters
b=3.0
N=100
T=100

# Coordinates arrays
x=np.zeros((T,N+1)); y=np.zeros((T,N+1)); z=np.zeros((T,N+1))

# Read simulation data
filename='simulation_FJC_b=%.1f_N=%d_T=%d.xyz'%(b,N,T)
with open(filename,'r') as f:
    for t in range(T):
        # First 2 lines unnecessary
        lines = f.readline()
        lines = f.readline()
        # Save coordinates and separate in x, y, z
        for i in range(N+1):
            lines = f.readline()
            coord = lines.split()
            x[t,i]=float(coord[1])
            y[t,i]=float(coord[2])
            z[t,i]=float(coord[3])

### Compute distance matrix
$$d_{i j} = ||\vec{R}_i - \vec{R}_j||

In [2]:
# Reshape arrays for broadcasting
x_t = x.reshape(T, N+1, 1)
y_t = y.reshape(T, N+1, 1)
z_t = z.reshape(T, N+1, 1)

# Compute squared differences
dx = (x_t - x_t.transpose(0, 2, 1)) ** 2
dy = (y_t - y_t.transpose(0, 2, 1)) ** 2
dz = (z_t - z_t.transpose(0, 2, 1)) ** 2

# Compute distance matrix
d = np.sqrt(dx + dy + dz)

## Structure factor

$$ I(k) = \sum_{i=0}^N \sum_{j=0}^N \left\langle \frac{\sin{k ||\vec{R}_i - \vec{R}_j||}}{k ||\vec{R}_i - \vec{R}_j||} \right\rangle$$

In [3]:
# Compute structure factor
k_grid = np.linspace(0, 0.1, 100)[1:]  # k=0 is not defined
k_grid = k_grid[:, np.newaxis, np.newaxis, np.newaxis]  # reshape for broadcasting

# Average over time
I_k = np.mean(np.sin(k_grid * d) / (k_grid * d), axis=1)

C:\Users\propietario\AppData\Local\Temp\ipykernel_11828\4194619670.py:6: RuntimeWarning: invalid value encountered in divide
  I_k = np.mean(np.sin(k_grid * d) / (k_grid * d), axis=1)


### Comparison with Guinier approximation
$$I(k) = (N + 1)^2 \left[1 - \frac{(k R_g)^2}{3} \right]$$

In [ ]:
# Compute Guinier approximation
Rg = np.sqrt(np.sum(d, axis=(1, 2)) / (6 * (N+1)))